# Digital Twin of Society - Education GER 📚

Schüler/-innen an allgemeinbildenden Schulen nach Klassen- bzw. Jahrgangsstufen, Bildungsbereichen und Ländern \
(Quelle: https://www.govdata.de/web/guest/suchen/-/details/schuler-innen-an-allgemeinbildenden-schulen-nach-klassen-bzw-jahrgangsstufen-bildungsbereichen--10)

Datenbereitsteller: Bundesministerium für Bildung und Forschung \
Veröffentlichende Stelle: Bundesministerium für Bildung und Forschung \
Deutsches Zentrum für Hochschul- und Wissenschaftsforschung

<br>
<br>
<table align="left">
<tr>
<td><img src="https://nuernberg.digital/fileadmin/system/NDF-Logo-Jahresneutral-RGB-black-keinRand.svg" width="100" /></td>
<td><img src="https://www.capgemini.com/de-de/wp-content/themes/capgemini-komposite/assets/images/logo.svg" /></td>
</tr>
</table>

## 1. Install requirements

In [ ]:
!pip install -q pandas geopandas tqdm folium

## 2. Imports

In [ ]:
import time
import pandas as pd
import warnings
import datetime
import folium
import branca.colormap as cm
from tqdm import tqdm
from folium.plugins import TimeSliderChoropleth
from folium.features import DivIcon
from IPython.display import clear_output

# 3. Data preprocessing

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Lade die Daten von https://www.govdata.de/web/guest/suchen/-/details/schuler-innen-an-allgemeinbildenden-schulen-nach-klassen-bzw-jahrgangsstufen-bildungsbereichen--1 in ein DataFrame. Tipp: Achte dabei auf Metadaten die nicht zum eigentlichen Datensatz gehören. Diese sollten nicht im DataFrame vorhanden sein.



In [ ]:
#@title Lösung
# load data
# df = pd.read_csv('https://raw.githubusercontent.com/Sultanow/dt_society/main/data/datagov/Allgemeinbildende-Schulen-nach-Bundeslaendern.csv', sep=';', skiprows=4, header=[0,1]) # skip intro text
# df = df[:-17] # cut of info text
# df

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Ziehe alle Bundesländer in eine separate Liste heraus und lasse dir die Liste ausgeben. Wir benötigen diese später noch. 


In [ ]:
#@title Lösung
# df.rename(columns={'Hessen 3)':'Hessen'}, inplace=True) # Rename hessen col header
# bundeslaender = df.columns.values
# bundeslaender = bundeslaender[4:]
# bundeslaender = [item[0] for item in bundeslaender] # remove "Anteile (%)"
# print(bundeslaender)

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

In den Daten sind Summenzeilen die Zwischenergebnisse aufsummieren.
Diese sind zwar hilfreich zur manuellen Betrachtung, für die automatisierte Verarbeitung aber eher störend. 

*   Entferne alle Informationen aus dem DataFrame die nicht direkt mit einer Jahrgangsstufe, dem Schuljahr und dem zugehörigen Bundesland in Zusammenhang stehen
*   Das DataFrame sollte nur noch die Überschriften "Schuljahr,	Jahrgangsstufe,	Abs. Anzahl gesamt,	Länder insgesamt,	Baden-Württemberg,	Bayern,	Berlin,	... besitzen.
*   Um später mit den Prozenzahlen weiterarbeiten zu können muss zudem das "," in einen "." konvertiert werden.



In [ ]:
#@title Lösung

# Behalte nur die Jahrgangsstufen (ohne Summen-Zeilen)
# df = df[[str(x).strip().isdigit() for x in df.iloc[:, 1]]]
# # Konvertiere "," zu "."
# df = df.replace({',': '.'}, regex=True)
# # Löse Multi-Level Index auf
# df.columns = df.columns.get_level_values(0)
# df = df.rename(columns={ df.columns[0]: "Schuljahr", df.columns[1]: "Jahrgangsstufe", df.columns[2]: "Abs. Anzahl gesamt" })
# df.head()

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Das DataFrame ist eine Tabelle mit zwei abhängigen Indicies. 
Dem Schuljahr und dem Bundesland. Beide zusammen lassen auf den %-Anteil schließen. Damit die weitere Verarbeitung einfacher wird, muss das DataFrame so transformiert werden, dass die Bundesländer mehrfach als Zeile aufgelistet sind mit dem jeweiligen Schuljahr und dem %-Anteil.

Aus der Struktur:

|index|Schuljahr|Jahrgangsstufe|Abs\. Anzahl gesamt|Länder insgesamt|Baden-Württemberg|Bayern|Berlin|Brandenburg|Bremen|Hamburg|Hessen|Mecklenburg-Vorpommern|Niedersachsen|Nordrhein-Westfalen|Rheinland-Pfalz|Saarland|Sachsen|Sachsen-Anhalt|Schleswig-Holstein|Thüringen|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|2|1999/2000|  1|825878|8\.2|9\.2|9\.3|6\.8|4\.3|8\.6|8\.5|9|4\.2|9\.3|8\.8|9\.3|9\.2|4\.7|4\.9|9\.4|4\.8|
|3|1999/2000|  2|864381|8\.6|9\.5|9\.5|7|5|8\.7|8\.9|9\.4|5\.1|9\.8|9\.1|9\.7|9\.5|5\.4|5\.6|10\.2|5\.3|

Sollte diese werden (Ausschnitt gekürzt):

|index|Schuljahr|Jahrgangsstufe|Abs\. Anzahl gesamt|Bundesland|Anteile in %|
|---|---|---|---|---|---|
|0|1999/2000|  1|825878|Baden-Württemberg|9\.2|
|1|1999/2000|  2|864381|Baden-Württemberg|9\.5|
|2|1999/2000|  3|916918|Baden-Württemberg|9\.8|
|3|1999/2000|  4|919245|Baden-Württemberg|9\.3|
|286|1999/2000|  1|825878|Bayern|9\.3|
|287|1999/2000|  2|864381|Bayern|9\.5|
|288|1999/2000|  3|916918|Bayern|9\.7|
|289|1999/2000|  4|919245|Bayern|9\.4|

**Tipp:** Die Transformation geht mit 2 Schleifen. Es gibt aber auch einen schnelleren Weg. Nutze hierfür die [Funktionen von Pandas DataFrame](https://pandas.pydata.org/docs/reference/general_functions.html).


In [ ]:
#@title Lösung
# df_melted = df.melt(id_vars=['Schuljahr', 'Jahrgangsstufe', 'Abs. Anzahl gesamt'], var_name="Bundesland", value_vars=bundeslaender, value_name="Anteile in %")
# df_melted

# Alternative Schleifen Lösung
#for row in df.values:
#    schuljahr_as_date = datetime.date(int(str(row[0]).split('/')[0]), 1, 1) # e.g. 2002-1-1 instead of 2002/2003
#    anzahl = row[3]
#    # Multi-Index auflösen und Flatten
#    for i in range(4, len(df.columns)): # 4 = erster Index fuer Bundesland...
#        anteile = float(row[i])
#        bundesland = df.columns[i]
#        jahrgangsstufe = int(row[1])
#        series = {'Schuljahr' : row[0], 'Jahrgangsstufe' : jahrgangsstufe, 'Abs. Anzahl gesamt' : anzahl, 'Bundesland' : bundesland, 'anteile in %' : anteile}
#        df_processed = df_processed.append(series, ignore_index=True)

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Korrektur der Datentypen:

*  Lasse dir die Datentypen für jeden Spalte ausgeben (**Tipp:** `df.dtypes`)
*  Konvertiere die Datentypen in das richtige Format:
    * Schuljahr = String
    * Jahrgangsstufe = int
    * Abs. Anzahl gesamt = int
    * Anteile in % = float

In [ ]:
#@title Lösung
# Ausgabe
# print(df_melted.dtypes)
# # Konvertierung
# df_melted = df_melted.astype({"Schuljahr": str, "Jahrgangsstufe" : int, "Abs. Anzahl gesamt" : int, "Anteile in %" : float}) 
# # Ausgabe
# print("===================================== \n", df_melted.dtypes)

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Damit der TimeSeriesSlider später richtig arbeiten kann, muss das Schuljahr in ein Datumsformat konvertiert werden. 
*  Konvertiere für jede Zeile des DataFrames das Schuljahr in die Form 2002-01-01. Wenn das Schuljahr z. B. 2002/2003 ist dann sollte daraus 2002-01-01 werden. 
*  Achte darauf, dass der Datentyp date ist.
*  Schreibe die Konveriterung direkt wieder ins DataFrame zurück (Ersetzen der Schreibweise: 2002/2003)

In [ ]:
#@title Lösung
# df_melted['Schuljahr'] = df_melted['Schuljahr'].apply(lambda x: datetime.date(int(str(x).split('/')[0]), 1, 1)) # e.g. 2002-1-1 instead of 2002/2003)
# df_melted.head()

# 4. Data visualization

Die GeoJson wird benötigt um die Grenzen auf der Karte zu definieren.

In [ ]:
# GeoJson für die Bundeslandgrenzen
geo_json_uri = f"https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/3_mittel.geo.json"

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Zunächst rendern wir eine Folium-Map für eine feste Jahrgangsstufe und ein festes Schuljahr. 

* Erstelle ein DataFrame (df_selection) der folgenden Form

|index|Bundesland|Anteile in %|
|---|---|---|
|51|Baden-Württemberg|2\.0|
|337|Bayern|1\.8|
|623|Berlin|3\.4|

* Du kannst dabei eine beliebige Jahrgangsstufe für ein Jahr wählen.



In [ ]:
#@title Lösung
# df_selection = df_melted[df_melted['Schuljahr'] == datetime.date(2002,1,1)]
# df_selection =  df_selection[df_selection['Jahrgangsstufe'] == 13]
# df_selection = df_selection[['Bundesland', 'Anteile in %']]

# df_selection.head()

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Rendere nun die Folium Map. Hilfestellung unter https://python-visualization.github.io/folium/quickstart.html

Verwende das Plugin folium.Choropleth zu farblichen Abstufung der Bundeslädnder.

In [ ]:
#@title Lösung
# fmap = folium.Map(
#   location=[51.164, 10.454], 
#   zoom_start=6.25,
#   tiles="cartodb positron",
#   min_zoom=6.25,
#   max_zoom=7
# )

# folium.Choropleth(
#     geo_data=geo_json_uri,
#     name="choropleth",
#     data=df_selection,
#     key_on="feature.properties.name",
#     columns=df_selection.columns,
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Unemployment Rate (%)",
# ).add_to(fmap)

# fmap

# 5. Interaktive Visualisierung

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Nachdem wir nun eine Map für einen festen Wertebereich rendern können, geht es  an die Interaktionsmöglichkeiten. Um die Schuljahre durchlaufen zu können kann das Plugin TimeSliderChoropleth verwendet werden. 


*  Baue eine Funktion die es erlaubt eine Jahrgangsstufe aus dem DataFrame zu selektieren. Gebe diese Werte als Pandas.Series Objekt zurück.

In [ ]:
#@title Lösung
# def selectJahrgangsstufe(jahrgangsstufe : int):
#     return df_melted[df_melted['Jahrgangsstufe'] == jahrgangsstufe].copy()

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Für die dynamische Visualisierung benötigen wir eine eigene Color-Map (Dies hat zuvor das Plugin folium.Choropleth automatisch übernommen)

Diese sieht für einen Wertebereich z. B. so aus: \\
<img src="https://i.postimg.cc/RhFGLT51/image.png" width="300"/>

*  Nutze hierfür die [Bibliothek branca und dort cmap](https://python-visualization.github.io/branca/colormap.html). \
*  Verpacke die Generierung in eine eigene Funktion. \


In [ ]:
#@title Lösung
# def generateCmap(df):
#   max_colour = max(df['Anteile in %'])
#   min_colour = min(df['Anteile in %'])
#   cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
#   df['colour'] = df['Anteile in %'].map(cmap)
#   cmap.caption = "education"
#   return cmap
  
# # Beispiel
# generateCmap(df_selection)

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Folium benötigt zum Darstellen ein Style Dictionary. \
Beispiel für ein Style-Dict:
```json
styledict = {
    '0': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        },
    ...,
    'n': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        }
}
```


Diese Funktion geben wir hier einmal vor.

In [ ]:
# """
#   build stlye dict
#   styledict: dict
#       A dictionary where the keys are the geojson feature ids and the values are
#       dicts of {time: style_options_dict}
# """
# def buildStyleDict(df):
#   country_idx = range(len(bundeslaender))

#   style_dict = {}
#   for i in country_idx:
#       country = bundeslaender[i]
#       result = df[df['Bundesland'] == country]
#       inner_dict = {}
#       for _, r in result.iterrows():
#           formatted_schuljahr = int(time.mktime((r['Schuljahr']).timetuple()))
#           inner_dict[formatted_schuljahr] = {'color': r['colour'], 'opacity': 0.7}
#       style_dict[str(i)] = inner_dict
#   return style_dict

In [ ]:
# df_selection = selectJahrgangsstufe(1) # Selektieren Jahrgangsstufe
# cmap = generateCmap(df_selection) # Generiere die zugehörige CMAP
# style_dict = buildStyleDict(df_selection) # Baue das Styledict

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

* Erstelle eine Map mit einem TimeSliderChoropleth.
* **Tipps**: 
  * Dieser benötigt das Styledict
  * Die CMAP muss der Folium-Map hinzugefügt werden um eine Skale zu besitzen

In [ ]:
#@title Lösung
# fmap = folium.Map(
#     location=[51.164, 10.454], 
#     zoom_start=6.25,
#     tiles="cartodb positron",
#     min_zoom=6.25,
#     max_zoom=7
# )

# TimeSliderChoropleth(
#     data=geo_json_uri,
#     styledict=style_dict,
#     overlay=False,
#     control=False,
# ).add_to(fmap)

# cmap.add_to(fmap)
# folium.LayerControl().add_to(fmap)

# fmap

<img src="https://cdn3.iconfinder.com/data/icons/miscellaneous-80/60/info-256.png" width="32" height="32">

Super, du hast es geschafft 😸\
Wenn du noch mehr Interaktion möchtest: \

* Überlege dir wie du mithilfe von ipywidgets ein Dropdown für die Jahrgangsstufen integrieren kannst.
* Gerne kannst du auch einmal mit Map-Marken experimentieren